In [ ]:
!pip install tensorflow
!pip install keras

In [3]:
#Importing Libraries
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio, display
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
import os

warnings.simplefilter("ignore")
np.random.seed(42)

In [4]:
#Loading Data
filepath = "/content/"

#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        
        all_midis.append(midi)



In [5]:
#Helping function
# notes[0] ---> Array of notes
# notes[1] ---> Array of chords

def extract_notes(files):
    notes = [[],[],[],[],[]]#[(notas,fuerza,fuerzaNormalizada,duración),...]
    chords = [[],[],[],[]]#[(notas,fuerza,fuerzaNormalizada,duración),...]
    pick = None
    for midi in files:
        songs = instrument.partitionByInstrument(midi)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):                    
                    notes[0].append(str(element.pitch))
                    notes[1].append(element.volume.velocity) #Fuerza
                    notes[2].append(element.volume.velocityScalar) #Fuerza Normalizado: Valor normalizado entre 0-1
                    notes[3].append(element.duration.type) #Duración :Music21 Durations are almost always measured in Quarter Notes, so an eighth note has a duration of 0.5.
                    notes[4].append(element.duration.quarterLength) #Duración :Music21 Durations are almost always measured in Quarter Notes, so an eighth note has a duration of 0.5.
                elif isinstance(element, chord.Chord):
                    chords[0].append(".".join(str(n) for n in element.normalOrder))
                    chords[1].append(element.volume.velocity) #Fuerza
                    chords[2].append(element.volume.velocityScalar) #Fuerza Normalizado: Valor normalizado entre 0-1
                    chords[3].append(element.duration.type) #Duración :Music21 Durations are almost always measured in Quarter Notes, so an eighth note has a duration of 0.5.
   
    return (notes,chords)

#Getting the list of notes as Corpus
Corpus = extract_notes(all_midis)

#DATOS EN SERIES TEMPORALES, En FORMATO ARRAY
notes = Corpus[0][0]
notes_fuerza = Corpus[0][1]
notes_fuerza_normalizada = Corpus[0][2]
notes_duracion = Corpus[0][3]

chords = Corpus[1][0]
chords_fuerza = Corpus[1][1]
chrod_fuerza_normalizada = Corpus[1][2]
chords_duracion = Corpus[1][3]


print("Total notes in all the Chopin midis in the dataset:", len(notes),"-",len(notes_duracion))

Total notes in all the Chopin midis in the dataset: 48038 - 48038


In [ ]:
print("First fifty values in the Notes:", notes[:50])
print("First fifty values in the Notes_Fuerza:", notes_fuerza[:50])
print("First fifty values in the Notes_fuerza_normalizada:", notes_fuerza_normalizada[:50])
print("First fifty values in the notes_duración:", notes_duracion[:50])


First fifty values in the Notes: ['B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-4', 'B-2', 'D5', 'E-5', 'B-4', 'E-3', 'E-5', 'F5', 'B-4', 'D3', 'F5', 'G5', 'E-3', 'E3', 'B-5', 'F3', 'C6', 'B-5', 'G#2', 'G#5', 'B4', 'B-2', 'B-5', 'G#5', 'G5', 'B-4', 'B-2', 'G#5', 'G5', 'F5', 'G#4', 'E-3', 'G5', 'F5', 'B-4']
First fifty values in the Notes_Fuerza: [80, 74, 70, 79, 75, 69, 78, 72, 67, 74, 69, 66, 73, 59, 59, 72, 40, 60, 49, 67, 40, 64, 48, 72, 44, 67, 52, 69, 70, 63, 36, 52, 48, 37, 67, 67, 58, 47, 44, 66, 66, 37, 59, 53, 64, 64, 28, 60, 49, 65]
First fifty values in the Notes_fuerza_normalizada: [0.6299212598425197, 0.5826771653543307, 0.5511811023622047, 0.6220472440944882, 0.5905511811023622, 0.5433070866141733, 0.6141732283464567, 0.5669291338582677, 0.5275590551181102, 0.5826771653543307, 0.5433070866141733, 0.5196850393700787, 0.5748031496062992, 0.4645669291338583, 0.4645669291338583, 0.5669291338582677, 0.31496062992125

In [ ]:

#First Lets write some functions that we need to look into the data
def show(music):
    display(Image(str(music.write("lily.png"))))
    
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

Melody_Snippet = chords_n_notes(notes[:100])

Melody_Snippet.show()
#show(Melody_Snippet)

SubConverterException: ignored

In [ ]:
#to play audio or corpus
print("Sample Audio From Data")
IPython.display.Audio("../input/music-generated-lstm/Corpus_Snippet.wav") 

Sample Audio From Data


NameError: ignored